In [93]:
library(testthat)

comm <- IRkernel::comm_manager()$new_comm('dc_project')
comm$open()
comm$send(list(a = 'hey'))

ProjectReporter <- R6::R6Class("ProjectReporter", inherit = testthat::ListReporter,
  public = list(
    all_tests = NULL,
    comm = NULL,
    initialize = function(..., comm) {
      super$initialize(...)
      self$comm <- comm
      self$all_tests = testthat:::Stack$new()
    },
    end_test = function(context, test) {
      out_env <- super$end_test(context, test)
      jsonable <- lapply(out_env$as_list(), `[`, c('message', 'test'))
      self$comm$send(jsonable)
      out_env
      #assign('output', out, envir=globalenv())
      
    }
  ),
  private = list(
  )
)

In [94]:
reporter <- ProjectReporter$new(comm = comm)
env = test_env()
reporter$start_file('some name')
with_reporter(reporter = reporter, start_end_reporter = TRUE,
    eval(
        {
            test_that("a passing test", {
                expect_equal(2, 2)
                expect_equal(3, 3)
            })
        
            test_that("a failing test", {
                expect_equal(2, 3)
                expect_equal(3, 4)
            })
        },
        envir = env
    )
)
